In [ ]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib 

In [6]:
import shutil
import cv2
import os
import mediapipe as mp
import numpy as np
import keyboard

In [3]:

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
IMAGE_PATH = os.path.join('IMG_DATA')

words = []
with open('Sample.txt', 'r') as file:
    # Read each line and store it in the list
    for line in file:
        words.append(line.strip())  # Remove leading and trailing whitespaces if needed

actions = np.array(words)   #words list
sequence_count = 30     #50 trials
sequence_length = 30    #30 frames / trial 


mp_pose = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils #drawing helpers
mp_holistic = mp.solutions.holistic     #holistic model


In [4]:
def folder_init():
#Create 30 folders per action
    for action in actions: 
        for sequence in range(sequence_count):
            try: 
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
                os.makedirs(os.path.join(IMAGE_PATH, action, str(sequence)))
            except:
                pass

def folder_delete():
    try:
        # Delete the entire directory tree
        shutil.rmtree(DATA_PATH)
        shutil.rmtree(IMAGE_PATH)
        print(f"Folder '{DATA_PATH}, {IMAGE_PATH}' and its subfolders deleted successfully.")
    except FileNotFoundError:
        pass


In [5]:

#recolors image and make detections
def recolor(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66),thickness=2, circle_radius=2))  #change display

    
def extract_keypoints(results): #convert landmakrs to nparray
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [9]:
folder_init()

In [10]:
# start webcam capture
cap = cv2.VideoCapture(1)
frame_rate = cap.get(cv2.CAP_PROP_FPS)

# initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # loop through actions
    for action in actions:
        for sequence in range(sequence_count):
            for frame_num in range(sequence_length):

                # read feed
                ret, frame = cap.read()

                # make detections
                image, results = recolor(frame, holistic)

                # draw landmarks
                draw_landmarks(image, results)
                
                # new Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # new Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # save frames as images
                img_path = os.path.join(IMAGE_PATH, action, str(sequence), f"frame{frame_num}.jpg")
                cv2.imwrite(img_path, image)


        # break if q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
                        
cap.release()
cv2.destroyAllWindows()


[ WARN:0@463.536] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@463.656] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
I0000 00:00:1706126594.311848  488791 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1706126594.316007  493667 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1), renderer: llvmpipe (LLVM 16.0.6, 256 bits)


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
